In [24]:
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
# Load data
LData = pd.read_csv("Laptop_Train_v2.csv")
RData = pd.read_csv("Restaurants_Train_v2.csv")

In [4]:
# Preprocess the lData
LData['text'] = LData['text'].apply(lambda x: x.lower()) # Convert Sentence to lowercase
LData['text'] = LData['text'].apply(lambda x: x.replace(".", "")) # Remove punctuation

LData['aspect'] = LData['aspect'].apply(lambda x: x.lower()) # Convert Sentence to lowercase
LData['aspect'] = LData['aspect'].apply(lambda x: x.replace(".", "")) # Remove punctuation
LData['aspect'] = LData['aspect'].apply(lambda x: x.replace('"', "")) # Remove punctuation

In [5]:
# Preprocess the Rdata
RData['text'] = RData['text'].apply(lambda x: x.lower()) # Convert Sentence to lowercase
RData['text'] = RData['text'].apply(lambda x: x.replace(".", "")) # Remove punctuation

RData['aspect'] = RData['aspect'].apply(lambda x: x.lower()) # Convert Sentence to lowercase
RData['aspect'] = RData['aspect'].apply(lambda x: x.replace(".", "")) # Remove punctuation
RData['aspect'] = RData['aspect'].apply(lambda x: x.replace('"', "")) # Remove punctuation

In [6]:
# Concatenate data from multiple domains
data = pd.concat([LData, RData])
# data = LData

In [7]:
#Delete the useless Columns 
data = data.drop(['id','from','to'], axis = 1)

In [8]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [9]:
data.insert(1, 'tags', "")


In [10]:
data['tags'] = data['text'].apply(lambda x: [token.tag_ for token in nlp(x)])

In [11]:
data['text'] = data['text'].apply(lambda x: [str(token.text) for token in nlp(x)])

In [12]:
## change the labels to integers for the training phase
data['polarity'][data["polarity"] == 'negative'] = 0
data['polarity'][data["polarity"] == 'neutral'] = 1
data['polarity'][data["polarity"] == 'positive'] = 2

In [13]:
data.drop(data[data.polarity == 'conflict'].index, inplace = True)

In [14]:
def get_polarity(aspect):
  return data.loc[data['aspect'] == aspect, 'polarity'].values[0]

In [15]:
data.head()

,text,tags,aspect,polarity
0,"[i, charge, it, at, night, and, skip, taking, ...","[PRP, VBP, PRP, IN, NN, CC, NN, VBG, DT, NN, I...",cord,1
1,"[i, charge, it, at, night, and, skip, taking, ...","[PRP, VBP, PRP, IN, NN, CC, NN, VBG, DT, NN, I...",battery life,2
2,"[the, tech, guy, then, said, the, service, cen...","[DT, NN, NN, RB, VBD, DT, NN, NN, VBZ, RB, VB,...",service center,0
3,"[the, tech, guy, then, said, the, service, cen...","[DT, NN, NN, RB, VBD, DT, NN, NN, VBZ, RB, VB,...",sales team,0
4,"[the, tech, guy, then, said, the, service, cen...","[DT, NN, NN, RB, VBD, DT, NN, NN, VBZ, RB, VB,...",tech guy,1


In [16]:
for index, row in tqdm(data.iterrows(), total=len(data)):
  tags = []
  tokens = list(data['text'][index])[0]
  for token in tokens:
    try:
      tags.append(get_polarity(token))
    except:
      tags.append(0)
  data.iloc[index]['tags'] = tags

  0%|          | 0/5818 [00:00<?, ?it/s]

In [17]:
data.head()

,text,tags,aspect,polarity
0,"[i, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",cord,1
1,"[i, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",battery life,2
2,"[the, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",service center,0
3,"[the, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",sales team,0
4,"[the, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",tech guy,1


In [18]:
final_data = data.copy()

In [19]:
for index, row in tqdm(final_data.iterrows(), total=len(final_data)):
  polarities = []
  tokens = list(final_data['text'][index])[0]
  for token in tokens:
    try:
      polarities.append(get_polarity(token))
    except:
      polarities.append(-1)
  final_data.iloc[index]['polarity'] = polarities

  0%|          | 0/5818 [00:00<?, ?it/s]

In [20]:
final_data.head()

,text,tags,aspect,polarity
0,"[i, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",cord,"[-1, 0, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1,..."
1,"[i, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",battery life,"[-1, 0, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1,..."
2,"[the, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",service center,"[-1, -1, 0, -1, -1, -1, 0, -1, -1, -1, -1, -1,..."
3,"[the, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",sales team,"[-1, -1, 0, -1, -1, -1, 0, -1, -1, -1, -1, -1,..."
4,"[the, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",tech guy,"[-1, -1, 0, -1, -1, -1, 0, -1, -1, -1, -1, -1,..."


In [21]:
#Delete the useless Columns 
final_data = final_data.drop(['aspect'], axis = 1)

In [22]:
final_data = final_data.rename(columns={'text': 'Tokens', 'tags': 'Tags', 'polarity': 'Polarities'})

In [23]:
final_data.to_csv('processed_data.csv', sep=',', index=False)